Import dependencies

In [1]:
import findspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *

Initiate with Spark for Windows

In [2]:
findspark.init("C:\\Spark\\spark-3.1.2-bin-hadoop2.7")

Create a Spark session

In [3]:
spark = SparkSession.builder.master("local").appName("PDP assignment 3").config("spark.executor.memory", "1gb").getOrCreate()
sc = spark.sparkContext

Load data file and show a brief preview of the first 10 rows

In [4]:
df = spark.read.format("CSV").option("header", "true").load("titanic.csv")
df.head(10)

[Row(Survived='0', Pclass='3', Name='Mr. Owen Harris Braund', Sex='male', Age='22', Siblings/Spouses Aboard='1', Parents/Children Aboard='0', Fare='7.25'),
 Row(Survived='1', Pclass='1', Name='Mrs. John Bradley (Florence Briggs Thayer) Cumings', Sex='female', Age='38', Siblings/Spouses Aboard='1', Parents/Children Aboard='0', Fare='71.2833'),
 Row(Survived='1', Pclass='3', Name='Miss. Laina Heikkinen', Sex='female', Age='26', Siblings/Spouses Aboard='0', Parents/Children Aboard='0', Fare='7.925'),
 Row(Survived='1', Pclass='1', Name='Mrs. Jacques Heath (Lily May Peel) Futrelle', Sex='female', Age='35', Siblings/Spouses Aboard='1', Parents/Children Aboard='0', Fare='53.1'),
 Row(Survived='0', Pclass='3', Name='Mr. William Henry Allen', Sex='male', Age='35', Siblings/Spouses Aboard='0', Parents/Children Aboard='0', Fare='8.05'),
 Row(Survived='0', Pclass='3', Name='Mr. James Moran', Sex='male', Age='27', Siblings/Spouses Aboard='0', Parents/Children Aboard='0', Fare='8.4583'),
 Row(Survi

Format the data into a proper dataframe

In [5]:
df = df.withColumn("Survived", df["Survived"].cast(IntegerType())) \
    .withColumn("Pclass", df["Pclass"].cast(IntegerType())) \
    .withColumn("Name", df["Name"].cast(StringType())) \
    .withColumn("Sex", df["Sex"].cast(StringType())) \
    .withColumn("Age", df["Age"].cast(IntegerType())) \
    .withColumn("Siblings/Spouses Aboard", df["Siblings/Spouses Aboard"].cast(IntegerType())) \
    .withColumn("Parents/Children Aboard", df["Parents/Children Aboard"].cast(IntegerType())) \
    .withColumn("Fare", df["Fare"].cast(FloatType()))

Question A:
Here i grouped the passengers by gender and class and calculated their respective probability of survival as a percentage

In [6]:
dfa = df.select("Survived", "Sex", "Pclass").toPandas()
dfa = dfa.groupby(["Sex", "Pclass"]).mean(["Survived"]) * 100
dfa

Survived
Sex    Pclass           
female 1       96.808511
       2       92.105263
       3       50.000000
male   1       36.885246
       2       15.740741
       3       13.702624

Question B: First i aquire all the children that were registered in 3rd class and under the age of 10, then i calculate how many children in this group survived the accident. Lastly i divide the number of survivors by the total number of children from this group and calculate their chances of survival.

In [10]:
dfb = df.select("Age", "Pclass", "Survived").toPandas()
dfb = dfb[(dfb['Age'] <= 10) & (dfb['Pclass'] == 3)][['Age', 'Survived', 'Pclass']]
survived = dfb[dfb['Survived'] == 1].count()[0]
chance = (survived / dfb.count()[0] * 100).item()
print("Chances of survival: " + str(chance))

Chances of survival: 41.509433962264154


Question C: Simple calculation of the average ticket price for each class using Pandas

In [8]:
dfc = df.select("Pclass", "Fare").toPandas()
dfc = dfc.groupby(["Pclass"]).mean(["Fare"])
dfc

,Fare
Pclass,
1,84.154686
2,20.662184
3,13.707707
